In [1]:
# импортируем необходимые библиотеки, функцию train_test_split()
# и классы StandardScaler, LogisticRegression, Pipeline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm.csv', sep=';')
data.head(3)

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,18975.456110,65999,237,1,14,0,6,0
1,4715.321344,0,65,19,56,0,3,0
2,5018.885233,54500,63,28,17,0,6,0


In [3]:
# разбиваем данные на обучающие и тестовые: 
# получаем обучающий массив признаков, тестовый 
# массив признаков, обучающий массив меток, 
# тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

In [4]:
# создаем конвейер - экземпляр класса Pipeline
pipe = Pipeline([('scaler', StandardScaler()), 
                 ('logreg', LogisticRegression(
                     solver='lbfgs', max_iter=200))])

In [5]:
# импортируем функцию make_pipeline()
from sklearn.pipeline import make_pipeline
# создаем с ее помощью конвейер
pipe_quick = make_pipeline(
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=200))

In [6]:
# смотрим этапы конвейера
pipe_quick.steps

[('standardscaler', StandardScaler()),
 ('logisticregression', LogisticRegression(max_iter=200))]

In [7]:
# обучаем конвейер
pipe.fit(X_train, y_train)
# оцениваем качество конвейера на обучающих данных
print("Правильность на обучающей выборке: {:.3f}".format(
    pipe.score(X_train, y_train)))
# оцениваем качество конвейера на тестовых данных
print("Правильность на тестовой выборке: {:.3f}".format(
    pipe.score(X_test, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.900


In [8]:
# извлекаем константу
intercept = np.round(pipe.named_steps['logreg'].intercept_[0], 3)
intercept

-2.205

In [9]:
# извлекаем коэффициенты
coef = np.round(pipe.named_steps['logreg'].coef_, 3)
coef

array([[-0.022,  0.042,  0.079, -0.056, -0.004, -0.017, -0.109]])

In [10]:
# записываем названия предикторов
feat_labels = X_train.columns

In [11]:
# печатаем название "Константа"    
print("Константа:", intercept)
# печатаем название "Регрессионные коэффициенты"
print("Регрессионные коэффициенты:")
# для удобства сопоставим каждому названию 
# предиктора соответствующий коэффициент
for c, feature in zip(coef[0], feat_labels):
    print(feature, c)

Константа: -2.205
Регрессионные коэффициенты:
Customer Lifetime Value -0.022
Income 0.042
Monthly Premium Auto 0.079
Months Since Last Claim -0.056
Months Since Policy Inception -0.004
Number of Open Complaints -0.017
Number of Policies -0.109
